In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [12]:
%matplotlib inline
plt.style.use('ggplot')
# font = {'weight': 'bold',
#         'size':   16}
# plt.rc('font', **font)

In [82]:
df_courses = pd.read_csv('data/courses.csv')

In [4]:
df_assessments = pd.read_csv('data/assessments.csv')

In [5]:
df_vle = pd.read_csv('data/vle.csv')

In [6]:
df_studentInfo = pd.read_csv('data/studentInfo.csv')

In [7]:
df_studentRegistration = pd.read_csv('data/studentRegistration.csv')

In [8]:
df_studentAssessment = pd.read_csv('data/studentAssessment.csv')

In [9]:
df_studentVle = pd.read_csv('data/studentVle.csv')

## Initial Approach
Since the schema contains 7 data frames, I want to look into each of them one-by-one to get a better understanding of them to then be able to make hypothesis and generate ideas of what data frame I need to build ahead of any modeling. I also want to hone in on the dependent variable(s) I want to end up creating prediction models for.

I've listed each raw Data Frame below and the questions I had after reading the documentation (https://analyse.kmi.open.ac.uk/open_dataset)

### 1 - Courses 
* How many modules are there?
* How many presentations per module?
* What is the relationship b/w modules and presentations
* What the average length of each presentation? Is it unique to the presentation or to the module?
* What is the detail behind the CCC, EEE, and GGG modules they mention in the description

In [83]:
df_courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [84]:
# Data description says "B" is for Feb and "J" is for module-presentation which started in Oct

In [85]:
months_ = df_courses.code_presentation.apply(lambda x: x[-1])
months_ = months_.apply(lambda x: 2 if x =='B' else 10)

In [86]:
years_ = df_courses.code_presentation.apply(lambda x: int(x[0:4]))

In [87]:
df_courses['month'] = months_
df_courses['year'] = years_
df_courses['day'] = 1

In [88]:
df_courses['date_time']  = pd.to_datetime(df_courses[['year', 'month', 'day']])

In [89]:
df_courses.drop(['month','year','day'], axis = 1, inplace = True)

In [90]:
df_courses.head()

,code_module,code_presentation,module_presentation_length,date_time
0,AAA,2013J,268,2013-10-01
1,AAA,2014J,269,2014-10-01
2,BBB,2013J,268,2013-10-01
3,BBB,2014J,262,2014-10-01
4,BBB,2013B,240,2013-02-01


In [91]:
df_courses.head(22)

,code_module,code_presentation,module_presentation_length,date_time
0,AAA,2013J,268,2013-10-01
1,AAA,2014J,269,2014-10-01
2,BBB,2013J,268,2013-10-01
3,BBB,2014J,262,2014-10-01
4,BBB,2013B,240,2013-02-01
5,BBB,2014B,234,2014-02-01
6,CCC,2014J,269,2014-10-01
7,CCC,2014B,241,2014-02-01
8,DDD,2013J,261,2013-10-01
9,DDD,2014J,262,2014-10-01


In [98]:
255/30

8.5

In [93]:
df_courses.sort_values('date_time',inplace = True)

In [94]:
df_courses = df_courses.reset_index(drop = True)

In [95]:
df_courses

,code_module,code_presentation,module_presentation_length,date_time
0,DDD,2013B,240,2013-02-01
1,FFF,2013B,240,2013-02-01
2,BBB,2013B,240,2013-02-01
3,GGG,2013J,261,2013-10-01
4,FFF,2013J,268,2013-10-01
5,EEE,2013J,268,2013-10-01
6,DDD,2013J,261,2013-10-01
7,AAA,2013J,268,2013-10-01
8,BBB,2013J,268,2013-10-01
9,DDD,2014B,241,2014-02-01


In [96]:
df_courses.groupby('date_time').count()

,code_module,code_presentation,module_presentation_length
date_time,,,
2013-02-01,3,3,3
2013-10-01,6,6,6
2014-02-01,6,6,6
2014-10-01,7,7,7


In [97]:
df_courses.describe()

,module_presentation_length
count,22.000000
mean,255.545455
std,13.654677
min,234.000000
25%,241.000000
50%,261.500000
75%,268.000000
max,269.000000


In [101]:
df_courses.groupby('code_module').describe()

module_presentation_length                                        \
                                 count        mean        std    min     25%   
code_module                                                                    
AAA                                2.0  268.500000   0.707107  268.0  268.25   
BBB                                4.0  251.000000  16.532796  234.0  238.50   
CCC                                2.0  255.000000  19.798990  241.0  248.00   
DDD                                4.0  251.000000  12.138094  240.0  240.75   
EEE                                3.0  259.333333  15.885003  241.0  254.50   
FFF                                4.0  254.500000  16.176114  240.0  240.75   
GGG                                3.0  257.000000  14.422205  241.0  251.00   

                                   
               50%     75%    max  
code_module                        
AAA          268.5  268.75  269.0  
BBB          251.0  263.50  268.0  
CCC          255.0  262.00  269.0  
DDD          251.0  261.25  262.0  
EEE          268.0  268.50  269.0  
FFF          254.5  268.25  269.0  
GGG          261.0  265.00  269.0

* 7 Total Modules
* data covers 4 "semesters" - Feb 2013, Oct 2013, Feb 2014, Oct 2014
* Each Module has 2-4 presentations, all of varying presentation length, but avg is 255 days (8.5mos)
* Modules - BBB, DDD, and FFF were offered each semester.
* Modules - AAA, CCC were offered in only 2 semesters
* Modules -- EEE, GGG were offered in 3 semesters
* Reason they list CCC, EEE, and GGG modules is due to them being "uneven" offered in 3 out of 4 semesters, or in CC's case, offerred in consecutive semesters, rather than in same one every yr like AAA.

### 2 - Assessments
* How many assessments are there per presentation?
* Are there always the same assessments per module?
* Does the Date - i.e. number of days from the start of the module-presentation till the final presentation date tell us the length of each module-presentation?
* What weights add up to 100% per each module-presentation? Is the Final Exam 100% and then TMA+CMA = 100%?

In [102]:
df_assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [103]:
df_assessments.describe()

,id_assessment,date,weight
count,206.000000,195.000000,206.000000
mean,26473.975728,145.005128,20.873786
std,10098.625521,76.001119,30.384224
min,1752.000000,12.000000,0.000000
25%,15023.250000,71.000000,0.000000
50%,25364.500000,152.000000,12.500000
75%,34891.750000,222.000000,24.250000
max,40088.000000,261.000000,100.000000


In [108]:
df_assessments[df_assessments.code_module == 'AAA']

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
5,AAA,2013J,1757,Exam,NaN,100.0
6,AAA,2014J,1758,TMA,19.0,10.0
7,AAA,2014J,1759,TMA,54.0,20.0
8,AAA,2014J,1760,TMA,117.0,20.0
9,AAA,2014J,1761,TMA,166.0,20.0


In [111]:
df_assessments[df_assessments.code_module == 'BBB']

,code_module,code_presentation,id_assessment,assessment_type,date,weight
12,BBB,2013B,14991,CMA,54.0,1.0
13,BBB,2013B,14992,CMA,89.0,1.0
14,BBB,2013B,14993,CMA,124.0,1.0
15,BBB,2013B,14994,CMA,159.0,1.0
16,BBB,2013B,14995,CMA,187.0,1.0
17,BBB,2013B,14984,TMA,19.0,5.0
18,BBB,2013B,14985,TMA,47.0,18.0
19,BBB,2013B,14986,TMA,89.0,18.0
20,BBB,2013B,14987,TMA,124.0,18.0
21,BBB,2013B,14988,TMA,159.0,18.0


In [112]:
df_assessments[df_assessments.code_module == 'DDD']

,code_module,code_presentation,id_assessment,assessment_type,date,weight
74,DDD,2013B,25341,CMA,23.0,2.0
75,DDD,2013B,25342,CMA,51.0,3.0
76,DDD,2013B,25343,CMA,79.0,3.0
77,DDD,2013B,25344,CMA,114.0,4.0
78,DDD,2013B,25345,CMA,149.0,4.0
79,DDD,2013B,25346,CMA,170.0,3.0
80,DDD,2013B,25347,CMA,206.0,6.0
81,DDD,2013B,25334,TMA,25.0,7.5
82,DDD,2013B,25335,TMA,53.0,10.0
83,DDD,2013B,25336,TMA,81.0,12.5


In [110]:
df_assessments.groupby(['code_module','code_presentation']).count()['id_assessment']

code_module  code_presentation
AAA          2013J                 6
             2014J                 6
BBB          2013B                12
             2013J                12
             2014B                12
             2014J                 6
CCC          2014B                10
             2014J                10
DDD          2013B                14
             2013J                 7
             2014B                 7
             2014J                 7
EEE          2013J                 5
             2014B                 5
             2014J                 5
FFF          2013B                13
             2013J                13
             2014B                13
             2014J                13
GGG          2013J                10
             2014B                10
             2014J                10
Name: id_assessment, dtype: int64

In [113]:
df_assessments.groupby(['code_module','code_presentation']).sum()['weight']

code_module  code_presentation
AAA          2013J                200.0
             2014J                200.0
BBB          2013B                200.0
             2013J                200.0
             2014B                200.0
             2014J                200.0
CCC          2014B                300.0
             2014J                300.0
DDD          2013B                200.0
             2013J                200.0
             2014B                200.0
             2014J                200.0
EEE          2013J                200.0
             2014B                200.0
             2014J                200.0
FFF          2013B                200.0
             2013J                200.0
             2014B                200.0
             2014J                200.0
GGG          2013J                100.0
             2014B                100.0
             2014J                100.0
Name: weight, dtype: float64

In [114]:
df_assessments[df_assessments.code_module == 'GGG']

,code_module,code_presentation,id_assessment,assessment_type,date,weight
176,GGG,2013J,37418,CMA,229.0,0.0
177,GGG,2013J,37419,CMA,229.0,0.0
178,GGG,2013J,37420,CMA,229.0,0.0
179,GGG,2013J,37421,CMA,229.0,0.0
180,GGG,2013J,37422,CMA,229.0,0.0
181,GGG,2013J,37423,CMA,229.0,0.0
182,GGG,2013J,37415,TMA,61.0,0.0
183,GGG,2013J,37416,TMA,124.0,0.0
184,GGG,2013J,37417,TMA,173.0,0.0
185,GGG,2013J,37424,Exam,229.0,100.0


In [115]:
df_assessments[df_assessments.code_module == 'CCC']

,code_module,code_presentation,id_assessment,assessment_type,date,weight
54,CCC,2014B,24286,CMA,18.0,2.0
55,CCC,2014B,24287,CMA,67.0,7.0
56,CCC,2014B,24288,CMA,137.0,8.0
57,CCC,2014B,24289,CMA,207.0,8.0
58,CCC,2014B,24282,TMA,32.0,9.0
59,CCC,2014B,24283,TMA,102.0,22.0
60,CCC,2014B,24284,TMA,151.0,22.0
61,CCC,2014B,24285,TMA,200.0,22.0
62,CCC,2014B,24290,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0


* All Presentations have the same amount of Assessments every semesters EXCEPT for BBB - 2014J (half the amount of assessments due to removing CMA assessments) and DDD - 2013B (double the amount of assessments due to having CMA assessments). 
* Weights always equal 200 for all presentations except for GGG (weight = 100. only its final exam carries any wieght and for CCC which has two final exams (weight =300)
* Dates tell you at what pt in presentation was the assessment held. Could add feature of % of class completed/progress....

### 3 - VLE
* Does the number of total VLE belong to a presentation or to a module?
* How many different types of VLEs are there per module-presentation?
* What is the **activity type** field contain. Descripton says it's the, "role associated with the module materials."

### 4 - studentInfo
* Do students have multiple rows in this DF depending on each module-presentation they've participated in?
* How many Final Scores are there per student?
* Is there any correlation between the 6 demographic vars (gender, region, highest_education, imd_band, age_band, disability) and Final Scores?
* Is there any correlation b/w the num_previous_attempts and studied_credits and Final Scores?
* Could there be any leakage in the data with students who previously took the module-presentation?

### 5 - studentRegistration
* What is the avg number of days to register per student?
* How many students unregistered from a module-presentation?
* What is the % of registered/Total Registrations and also unregistered/Total Registrations?

### 6 - studentAssessment
* How many on-time vs. late submissions are there?
* What is the average score split by on-time and late submissions?
* What % of assessments are "banked" i.e. if the assessment result was transferred from a previous presentation/
* What pct is Num Fails (<40 Score) to total Assessments

### 7 - studentVle
* Can students have multiple rows per a singular Vle due to tracking their interaction with it by day?
* How many clicks are there per Student per VLE
* How many clicks per student per day are there per VLE
* Plot Usage vs. Date